<a href="https://colab.research.google.com/github/MuratCKoc/salesinsider/blob/main/ProphetsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from fbprophet import Prophet
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/MuratCKoc/salesinsider/main/salesinsider/data/Groceries_dataset.csv')
data_df = df[['Date', 'itemDescription']]
data_df.head()

,Date,itemDescription
0,21-07-2015,tropical fruit
1,05-01-2015,whole milk
2,19-09-2015,pip fruit
3,12-12-2015,other vegetables
4,01-02-2015,whole milk


In [31]:
cat_df = pd.get_dummies(data_df["itemDescription"])
final_df = pd.concat([data_df,cat_df],axis=1)
grouped_df = final_df.groupby("Date", as_index=False).sum()

grouped_df.head()

,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,01-01-2014,2,0,0,0,0,0,0,0,0,...,0,0,2,2,0,0,0,2,4,0
1,01-01-2015,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,0,3,0,0
2,01-02-2014,0,0,0,0,0,0,0,0,2,...,0,0,2,1,0,1,0,1,3,0
3,01-02-2015,0,1,0,1,0,0,0,0,2,...,0,0,1,0,0,1,0,9,6,0
4,01-03-2014,0,0,0,0,0,0,0,0,0,...,0,0,1,3,0,1,0,1,1,0


In [34]:
columns = grouped_df.columns
print (len(columns))

168


In [37]:
master_df = pd.DataFrame()
predictions_records = dict()

In [40]:
 for j in range(727, len(grouped_df.index)):
   # Predict thru all transactions.
  for i in range (165, len(grouped_df.columns)): 
    prophet_df = grouped_df[["Date", grouped_df.columns[i]]]
    prophet_df.columns = ["ds", 'y']
    prophet_df['ds'] = pd.to_datetime(prophet_df.ds,format='%d-%m-%Y') 
    model = Prophet()
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=213)
    forecast = model.predict(future)
    predicted_df = forecast[["ds", "yhat"]]
    predicted_df.yhat = predicted_df.yhat.round()
    predicted_df.rename({})
    predicted_df = predicted_df.rename( columns={"ds": "Date", "yhat": grouped_df.columns[i]})

    #initialize container df
    if len(master_df.index) == 0:
      master_df = predicted_df.copy()
    else:
      master_df = pd.merge(master_df, predicted_df, on="Date")
    
    if i == len(grouped_df.columns):
      break
  #save the records
  #predictions_records.push(master_df.to_dict('records'))
  master_df.to_csv("salesinsider/data/prophet1.csv")
  master_df.to_json(r'salesinsider/data/prophet1.json', orient='records')
  print(i)

/Users/mkc/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/mkc/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/mkc/anaconda/lib/python3.6/site-packa

In [43]:
predictions_list  = master_df.set_index('Date').to_dict('list')

In [46]:
 predictions_records = master_df.to_dict('records')